## Demo Notebook for Accessing PV Rooftop Data on Azure

The National Renewable Energy Laboratory's (NREL) PV Rooftop Database (PVRDB) is a lidar-derived, geospatially-resolved dataset of suitable roof surfaces and their PV technical potential for 128 metropolitan regions in the United States. The source lidar data and building footprints were obtained by the U.S. Department of Homeland Security Homeland Security Infrastructure Program for 2006-2014. Using GIS methods, NREL identified suitable roof surfaces based on their size, orientation, and shading parameters Gagnon et al. (2016). Standard 2015 technical potential was then estimated for each plane using NREL's System Advisory Model.

This notebook will demonstrate how to access the PV Rooftop data located in Azure BLOB storage.

### Get Access Token

You do not need an Azure account to access public data. Instead, you can obtain a temporary access token via the Planetary Computer's API. This can be accomplished via either the requests or planetary_computer libraries. Both options are shown below.

In [ ]:
# get a token with requests
import requests

token = requests.get(
    'https://planetarycomputer.microsoft.com/api/sas/v1/token/nrel/oedi'
).json()['token']

In [ ]:
# get a token with planetary-computer
import planetary_computer

token = planetary_computer.sas.get_token('nrel', 'oedi').token


### Explore Container

First, we use the token to create a PyFileSystem object. We can then use ParquetDataset objects to explore the metadata for each table. pv_rooftop consists of 4 tables:
- buildings
- aspects
- developable-planes
- rasd

Each table is partitioned by city_year.

In [ ]:
from pyarrow.fs import PyFileSystem, FSSpecHandler
from adlfs import AzureBlobFileSystem
import pyarrow.parquet as pq

# Create file system using token
fs = PyFileSystem(
    FSSpecHandler(
        AzureBlobFileSystem('nrel', credential=token)
    )
)

# Create ParquetDataset for the buildings table
buildings_dataset = pq.ParquetDataset('oedi/pv-rooftop/buildings', filesystem=fs)

# View the partition keys
city_years = buildings_dataset.partitioning.dictionaries
city_years


In [ ]:
# View the schema for the buildings table
buildings_dataset.schema

### Read Data

pv_rooftop is a large data set. For the purposes of this example, we will read data from a single partition, city_year=albany_ny_13, and take a random sample of 100 buildings. We will read the tables directly into geodataframes. This may take several minutes.

In [ ]:
import pandas as pd
import geopandas as gpd

# Read the bldg_fid column from the buildings table and take a random sample of 100 buildings.
bldg_fid_sample = pd.read_parquet(
    'oedi/pv-rooftop/buildings',
    filesystem=fs,
    filters=[('city_year', '=', 'albany_ny_13')],
    columns=['bldg_fid']
).sample(100)['bldg_fid']

In [ ]:
# Read buildings table using bldg_fid_sample as a filter
buildings = gpd.read_parquet(
    'oedi/pv-rooftop/buildings',
    filesystem=fs,
    filters=[
        ('city_year', '=', 'albany_ny_13'),
        ('bldg_fid', 'in', bldg_fid_sample)
    ],
    columns=['gid', 'city', 'state', 'year', 'bldg_fid', 'the_geom_4326']
)

In [ ]:
# Read aspects table using bldg_fid_sample as a filter
aspects = gpd.read_parquet(
    'oedi/pv-rooftop/aspects',
    filesystem=fs,
    filters=[
        ('city_year', '=', 'albany_ny_13'),
        ('bldg_fid', 'in', bldg_fid_sample)
    ],
    columns=['gid', 'city', 'state', 'year', 'bldg_fid', 'aspect', 'the_geom_4326']
)

# Add a column for the aspect_string
aspect_lookup = {
    0: 'flat',
    1: 'north',
    2: 'northeast',
    3: 'east',
    4: 'southeast',
    5: 'south',
    6: 'southwest',
    7: 'west',
    8: 'northwest'
}
aspects['aspect_string'] = aspects['aspect'].replace(aspect_lookup)

In [ ]:
# Read developable-planes table using bldg_fid_sample as a filter
developable_planes = gpd.read_parquet(
    'oedi/pv-rooftop/developable-planes',
    filesystem=fs,
    filters=[
        ('city_year', '=', 'albany_ny_13'),
        ('bldg_fid', 'in', bldg_fid_sample)
    ],
    columns=['gid', 'city', 'state', 'year', 'bldg_fid', 'footprint_m2', 'slope', 'flatarea_m2', 'slopeconversion', 'slopearea_m2', 'aspect', 'the_geom_4326']
)

# Add a column for the aspect_string
developable_planes['aspect_string'] = developable_planes['aspect'].replace(aspect_lookup)

In [ ]:
# Read rasd table
rasd = gpd.read_parquet(
    'oedi/pv-rooftop/rasd',
    filesystem=fs,
    filters=[
        ('city_year', '=', 'albany_ny_13')
    ],
    columns=['gid', 'city', 'state', 'year', 'the_geom_4326']
)

### Visualize Data

We are now ready to visualize the data using folium.

In [ ]:
import folium

# Dictionary for coloring the polygons based on aspect
color_dict = {
    'flat':      'yellow',
    'north':     'red',
    'northeast': 'red',
    'east':      'yellow',
    'southeast': 'green',
    'south':     'green',
    'southwest': 'green',
    'west':      'yellow',
    'northwest': 'red'
}
color = aspects['aspect_string'].replace(color_dict)
m = buildings.explore(color='gray', name='buildings')
m = aspects.explore(m=m, name='aspects', color=color)
m = developable_planes.explore(m=m, name='developable-planes', color=color)
m = rasd.explore(m=m, name='rasd')
folium.LayerControl().add_to(m)
m

### Export Data

There are many options for exporting the data for use in GIS software. Here, we demonstrate writing a geopackage.

In [ ]:
file_name = 'pv_rooftop_albany_ny_13.gpkg'
buildings.to_file(file_name, layer='buildings', driver="GPKG")
aspects.to_file(file_name, layer='aspects', driver="GPKG")
developable_planes.to_file(file_name, layer='developable-planes', driver="GPKG")
rasd.to_file(file_name, layer='rasd', driver="GPKG")